In [14]:
import numpy as np

In [15]:
def norm_coordinates(v, normf):
    res = np.copy(v)
    res[0:2] = res[0:2]/normf
    return res

In [16]:
def norm_coordinates_multi(vs, normf):
    res = np.copy(vs)
    res[:, 0:2] = res[:, 0:2]/normf
    return res

In [17]:
def hom2eucl_multi(vs):
    assert(0 not in vs[:,2]), "None of the points must be at infinity!"
    return vs[:,0:2]/vs[:,2][:,None]

In [18]:
with open("checkers_fullsize/outs.csv") as inf:
    inrows = np.array([l.split(",") for l in inf.read().splitlines()]).astype("float")
vpxs, vpys, As, Bs, Cs, centers = inrows.reshape((12, -1, 3)).swapaxes(0,1)
normf = 1000
vpxs_en, vpys_en = norm_coordinates_multi(hom2eucl_multi(vpxs), normf), norm_coordinates_multi(hom2eucl_multi(vpys), normf)

In [19]:
# vpxs and vpys are (n,2)-arrays with one entry for each of the n pictures.
# Entries *must* be in Euclidean coordinates!
def compute_intrinsics_from_vps(vpxs, vpys):
    def compute_ABCD_i(vpmi, vpni, vpm1, vpn1):
        u_mi, v_mi = vpmi[0], vpmi[1]
        u_ni, v_ni = vpni[0], vpni[1]
        u_m1, v_m1 = vpm1[0], vpm1[1]
        u_n1, v_n1 = vpn1[0], vpn1[1]

        A_i = u_mi - u_m1 + u_ni - u_n1
        B_i = v_mi - v_m1 + v_ni - v_n1
        C_i = v_m1*v_n1 - v_mi*v_ni
        D_i = u_mi*u_ni - u_m1*u_n1
        return A_i, B_i, C_i, D_i
    A = []
    b = []
    vpx_0, vpy_0 = vpxs[0], vpys[0]
    for vpx, vpy in zip(vpxs[1:], vpys[1:]):
        A_i, B_i, C_i, D_i = compute_ABCD_i(vpx, vpy, vpx_0, vpy_0)
        A += [[A_i, B_i, C_i]]
        b += [D_i]
    u_0, alpha2v_0, alpha2 = np.linalg.lstsq(A, b,rcond=-1)[0]
    alpha = np.sqrt(alpha2)
    v_0 = alpha2v_0/alpha
    f = 0-((v_0 - vpx_0[1])*(v_0-vpy_0[1]))-(1/alpha2)*(vpx_0[0]-u_0)*(vpy_0[0]-u_0)
    return [u_0, v_0], alpha, f

In [20]:
compute_intrinsics_from_vps(vpxs_en, vpys_en)

([1.70552418744573, -1.1738396991609852],
 1.3587999182314354,
 12.103844935135335)